In [20]:
import numpy as np 
import matplotlib.pyplot as plt
from statistics import mean
from scipy import constants as cst
from scipy import signal
from scipy.stats import linregress
from scipy.signal import find_peaks, peak_prominences
from scipy.signal import freqz, lfilter, welch
from scipy import fftpack
from scipy.interpolate import interp1d
from mpl_toolkits.axes_grid1 import host_subplot
from mpl_toolkits import axisartist
import math
from scipy.constants import find, physical_constants, c
import datetime
import scipy.special
from math import factorial
import import_ipynb

In [21]:
def generate_grid(L, grid_size):
    return np.linspace(-L / 2, L / 2, grid_size + 1)


def generate_phi(phi0, r0, x):
    return -phi0 * np.exp(-x**2 / r0**2)

In [22]:
class CheckingCriterions():
    
    def __init__(self, phi, dl):
        self.phi = phi
        self.dl = dl # It's just the distance between adjacent grid points, dtheta or dxl depend on situation
        self.dphi_dl = np.gradient(self.phi, self.dl)

    
    def check_the_nyquist_criterion(self):
        nyquist_values = self.dl * np.abs(self.dphi_dl)
        
        if np.any(nyquist_values >= np.pi):
            print("Warning: Nyquist criterion isn't satisfied!")
        else:
            print("Nyquist criterion is satisfied.")
            

    def condition_for_minimizing_edge_effects(self, z_values, k, L):
        for zi in z_values:
            zTheta_values = zi * np.abs(self.dphi_dl)/k
        
            if np.any(zTheta_values > L):
                print("Warning! Edge effects aren't minimized!")
            else: 
                print("Edge effects are minimized")
    


In [23]:

# Phase Screen for Cartesian Coordinates
class PhaseScreenDec:
    def __init__(self, k, phi0, r0, grid_size, L, z_values):
        self.k = k
        self.phi0 = phi0
        self.r0 = r0
        self.grid_size = grid_size
        self.L = L
        self.dx = self.L/self.grid_size
        self.x = generate_grid(self.L, grid_size)
        self.z_values = z_values
        self.phi = generate_phi(phi0, r0, self.x)

    
    def generation_of_phase_screen_dec(self):
        
        #1st step
        U = np.ones(self.grid_size + 1, dtype=complex)
        U *= np.exp(1j * self.phi)
        
        #2nd step
        U_fft = np.fft.fftshift(np.fft.fft(U))
        K = 2 * np.pi * np.fft.fftshift(np.fft.fftfreq((self.grid_size + 1), d=self.dx))
        results = []

        for zi in self.z_values:
            
            H = np.exp(1j * zi * K**2 / (2 * self.k) + 1j * K * self.x)
            U_prop = np.exp(-1j * self.k * zi) * np.fft.ifft(np.fft.ifftshift(U_fft * H))
            E = U_prop*np.exp(-1j*self.k*zi)
            results.append(np.abs(E)**2)

        return results


In [24]:
# Theoretical Intensity
class E_theoretical:
    def __init__(self, k, phi0, r0, grid_size, L, z_values):
        self.k = k
        self.grid_size = grid_size
        self.L = L
        self.dx = self.L/self.grid_size
        self.x = generate_grid(self.L, grid_size)
        self.z_values = z_values
        self.phi0 = phi0
        self.r0 = r0
    

    def calculate_E_theoretical(self, max_n=170):
        I_theory = []

        for zi in self.z_values:
            summation = np.zeros_like(self.x, dtype=np.complex128)
            E_prefactor = (
                np.exp(1j * self.k * zi + 1j * self.k * self.x**2 / (2 * zi)) / np.sqrt(1j * self.k * zi)
            )

            for n in range(max_n):
                coef = ((1j * self.phi0) ** n) / factorial(n)
                denom = 2 * n / (self.k * self.r0) ** 2 - 1j / (self.k * zi)
                exp_arg = -self.x**2 / (2 * zi**2 * denom)
                summation += coef * denom**-0.5 * np.exp(exp_arg)

            I_theory.append(np.abs(E_prefactor * summation) ** 2)

        return I_theory


In [25]:
# Cylindrical Coordinates Phase Screen
class PhaseScreenCyl:
    def __init__(self, k, phi0, r0, grid_size, R, Grid_theta, r, dr_values):
        self.k = k
        self.grid_size = grid_size
        self.R = R
        self.r = r
        self.Grid_theta = Grid_theta
        self.dtheta = self.Grid_theta/self.grid_size
        self.theta = generate_grid(self.Grid_theta, grid_size)
        self.dr_values = dr_values 
        self.phi0 = phi0
        self.r0 = r0
        self.phi = generate_phi(self.phi0, self.r0, (self.theta * self.R))



    def generation_of_phase_screen_cyl(self):
        
        U_start = np.ones(self.grid_size + 1, dtype=complex)
        K = 2 * np.pi * np.fft.fftshift(np.fft.fftfreq((self.grid_size + 1), d=self.dtheta))
        results = []
        
        for dri in self.dr_values:
            #1st step
    
            U = np.exp(
                -1j * (self.phi - 1 / (2 * self.k) * (1 / self.r - 1 / (self.r + dri))) + np.log((self.r + dri)/self.r)
            )*U_start

            #2nd step
            U_fft = np.fft.fftshift(np.fft.fft(U))
            H = np.exp(-1j * K**2 / (2 * self.k) * (1 / self.r - 1 / (self.r + dri)))
            E_prop = (
                np.exp(1j * self.k * dri) / self.r * np.fft.ifft(np.fft.ifftshift(self.r*U_fft * H))
            )
            results.append(np.abs(E_prop) ** 2)

        return results
